In [75]:
import pandas as pd
from wandb import Api
from wandb_utils import retrieve_best_model_artifact
import shutil
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import json

api = Api()

In [41]:
root = Path("..")
model_dir = root / "models"
if not model_dir.exists():
    model_dir.mkdir()
    
def get_path(rmsd_threshold: int, split_type: str, split_fold: int, model: str) -> Path:
    p = model_dir / f"rmsd_cutoff_{rmsd_threshold}" / split_type / str(split_fold) / model
    if not p.exists():
        p.mkdir(parents=True)
    return p

get_path(2, "random-k-fold", 0, "CGNN")

PosixPath('../models/rmsd_cutoff_2/random-k-fold/0/CGNN')

In [42]:
eval_data = pd.read_csv("eval_data_final.csv")
eval_data = eval_data[eval_data["model"] != "LGNN"]
eval_data

,wandb_id,split_type,split_fold,rmsd_threshold,test/corr,test/mae,model
0,1abvnegf,random-k-fold,1,6,0.740718,0.642563,CGNN-3D
1,ko4x8wyq,pocket-k-fold,4,6,0.448779,0.945433,DTI
2,2iymo937,pocket-k-fold,4,4,0.359720,0.867832,DTI
3,3m7da3ej,pocket-k-fold,3,2,0.283482,0.928143,DTI
4,3hvgex5f,pocket-k-fold,2,2,0.313985,0.907610,DTI
...,...,...,...,...,...,...,...
170,3oapxub7,scaffold-k-fold,1,4,0.560074,0.788821,DTI
171,1qjju40e,scaffold-k-fold,0,4,0.512872,0.822356,DTI
172,1yhqbdfi,scaffold-k-fold,4,2,0.559146,0.760455,DTI
173,13quy6hs,scaffold-k-fold,0,2,0.528767,0.813634,DTI


In [49]:
for _, row in tqdm(eval_data.iterrows()):
    run_id = row["wandb_id"]
    
    run = api.run(f"nextaids/kinodata-docked-rescore/{run_id}")
    json_config = run.json_config
    artifact = retrieve_best_model_artifact(run)
    artifact_path = Path(artifact.download())
    
    target_path = get_path(
        row["rmsd_threshold"],
        row["split_type"],
        row["split_fold"],
        row["model"]
    )
    for subdir in target_path.iterdir():
        shutil.rmtree(subdir)
    
    shutil.move(artifact_path, target_path)
    with open(target_path / "config.json", "w") as f_config:
        f_config.write(json_config)

0it [00:00, ?it/s]wandb:   1 of 1 files downloaded.  
1it [00:00,  1.35it/s]wandb:   1 of 1 files downloaded.  
2it [00:01,  1.02it/s]wandb:   1 of 1 files downloaded.  
3it [00:02,  1.01s/it]wandb:   1 of 1 files downloaded.  
4it [00:03,  1.00it/s]wandb:   1 of 1 files downloaded.  
5it [00:04,  1.02s/it]wandb:   1 of 1 files downloaded.  
6it [00:06,  1.06s/it]wandb:   1 of 1 files downloaded.  
7it [00:07,  1.14s/it]wandb:   1 of 1 files downloaded.  
8it [00:09,  1.29s/it]wandb:   1 of 1 files downloaded.  
9it [00:10,  1.25s/it]wandb:   1 of 1 files downloaded.  
10it [00:11,  1.22s/it]wandb:   1 of 1 files downloaded.  
11it [00:12,  1.20s/it]wandb:   1 of 1 files downloaded.  
12it [00:13,  1.16s/it]wandb:   1 of 1 files downloaded.  
13it [00:15,  1.42s/it]wandb:   1 of 1 files downloaded.  
14it [00:16,  1.30s/it]wandb:   1 of 1 files downloaded.  
15it [00:17,  1.20s/it]wandb:   1 of 1 files downloaded.  
16it [00:18,  1.12s/it]wandb:   1 of 1 files downloaded.  
17it [00:19

In [80]:
import wandb
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joschka (nextaids). Use `wandb login --relogin` to force relogin


In [81]:
from kinodata.model.complex_transformer import ComplexTransformer
from kinodata.model.dti import DTIModel

model_cls = {
    "DTI": DTIModel,
    "CGNN": ComplexTransformer,
    "CGNN-3D": ComplexTransformer
}

p = get_path(2, "scaffold-k-fold", 0, "CGNN")
mp = p / "model.ckpt"

def load_from_checkpoint(
    rmsd_threshold,
    split_type,
    fold,
    model
):
    cls = model_cls[model]
    p = get_path(rmsd_threshold, split_type, fold, model)
    ckp_path = p / "model.ckpt"
    with open(p / "config.json", "r") as f_config:
        config = json.load(f_config)
    config = {key: value["value"] for key, value in config.items()}
    model = cls.load_from_checkpoint(ckp_path, **config)
    return model

In [ ]:
ComplexTransformer.load_from_checkpoint()

In [82]:
load_from_checkpoint(2, "scaffold-k-fold", 0, "CGNN")

AttributeError: 'dict' object has no attribute 'loss_type'